In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/stat940project')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

%reload_ext autoreload

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import os
import numpy as np
import random
import math
import torch.optim as optim
from tqdm.auto import tqdm, trange

In [ ]:
from dataset import load_data, save_data, load_targets, AdversarialDataset
from utils.augmentations import *
from utils.utils import run, set_seed, pcshow
from models import Pointnet, DGCNN
from attacks import DeepFool

In [ ]:
set_seed(1)

In [ ]:
parent_dir = '/content/drive/MyDrive/stat940project'
data_path = os.path.join(parent_dir, 'data/ModelNet40')

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')

else:
    device = torch.device('cpu')
    print('running on CPU')

test_X, test_y = load_data(data_path, mode='test')
test_targets = load_targets(data_path)



running on GPU


In [ ]:
default_transforms = transforms.Compose(
    [
        ToTensor()
    ]
)

In [ ]:
testset = AdversarialDataset(test_X, test_y, test_targets, transforms=default_transforms)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
def run(targeted, file_name, output_path, model_name, signed = False):
      if model_name == "pointnet":
          model = Pointnet(device=device).to(device)
      else:
          model = DGCNN().to(device)

      checkpoint_path = os.path.join(parent_dir, f'checkpoints/{model_name}/best.pt')
      model.load_state_dict(torch.load(checkpoint_path, map_location=device)["model_state_dict"])
      model.eval()

      all_adv_pc = []
      all_gt_labels = []
      all_target_labels = []
      all_success = 0

      attacker = DeepFool(model, max_iter=20, overshoot=0.02)

      for pc, labels, targets in tqdm(testloader):
          with torch.no_grad():
              pc = pc.to(device)
              labels = labels.to(device)


          adv_pc, success_num = attacker.attack(pc, labels, device)

          all_adv_pc += [adv_pc]
          all_gt_labels += [labels.detach().cpu().numpy()]
          all_target_labels += [targets.detach().cpu().numpy()]
          all_success += success_num
          print(f'{success_num}/{adv_pc.shape[0]} succesful attacks')

      print(f"Success Rate: {all_success / len(test_X)}")


      all_adv_pc = np.concatenate(all_adv_pc, axis=0)
      all_gt_labels = np.concatenate(all_gt_labels, axis=0)
      all_target_labels = np.concatenate(all_target_labels, axis=0)



      save_data(all_adv_pc, file_name, output_path)
      save_data(all_gt_labels, 'test_labels', output_path)
      save_data(all_adv_pc, 'target_labels', output_path)
      return all_adv_pc

In [ ]:
path = os.path.join(parent_dir, 'data/DeepFool/Pointnet')
if not os.path.exists(path):
    os.makedirs(path)
all_adv_pc = run(False, 'adv_test_untarget', path, 'pointnet', signed = False)

  0%|          | 0/39 [00:00<?, ?it/s]

61/64 succesful attacks
63/64 succesful attacks
61/64 succesful attacks
61/64 succesful attacks
62/64 succesful attacks
62/64 succesful attacks
64/64 succesful attacks
63/64 succesful attacks
62/64 succesful attacks
63/64 succesful attacks
61/64 succesful attacks
61/64 succesful attacks
63/64 succesful attacks
61/64 succesful attacks
60/64 succesful attacks
62/64 succesful attacks
62/64 succesful attacks
60/64 succesful attacks
62/64 succesful attacks
63/64 succesful attacks
62/64 succesful attacks
61/64 succesful attacks
63/64 succesful attacks
63/64 succesful attacks
64/64 succesful attacks
62/64 succesful attacks
63/64 succesful attacks
61/64 succesful attacks
63/64 succesful attacks
62/64 succesful attacks
62/64 succesful attacks
60/64 succesful attacks
64/64 succesful attacks
60/64 succesful attacks
59/64 succesful attacks
62/64 succesful attacks
64/64 succesful attacks
61/64 succesful attacks
36/36 succesful attacks
Success Rate: 0.9679902755267423


In [ ]:
x1, y1, z1 = all_adv_pc[1].T
x2, y2, z2 = test_X[1].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='blue')

In [ ]:
path = os.path.join(parent_dir, 'data/DeepFool/DGCNN')
if not os.path.exists(path):
    os.makedirs(path)
all_adv_pc = run(False, 'adv_test_untarget', path, 'dgcnn', signed = False)

  0%|          | 0/39 [00:00<?, ?it/s]

61/64 succesful attacks
62/64 succesful attacks
64/64 succesful attacks
64/64 succesful attacks
62/64 succesful attacks
62/64 succesful attacks
64/64 succesful attacks
63/64 succesful attacks
63/64 succesful attacks
63/64 succesful attacks
56/64 succesful attacks
63/64 succesful attacks
62/64 succesful attacks
63/64 succesful attacks
63/64 succesful attacks
61/64 succesful attacks
59/64 succesful attacks
63/64 succesful attacks
61/64 succesful attacks
62/64 succesful attacks
61/64 succesful attacks
60/64 succesful attacks
63/64 succesful attacks
62/64 succesful attacks
60/64 succesful attacks
54/64 succesful attacks
62/64 succesful attacks
61/64 succesful attacks
64/64 succesful attacks
64/64 succesful attacks
64/64 succesful attacks
62/64 succesful attacks
62/64 succesful attacks
60/64 succesful attacks
62/64 succesful attacks
58/64 succesful attacks
61/64 succesful attacks
61/64 succesful attacks
35/36 succesful attacks
Success Rate: 0.963128038897893


In [ ]:
x1, y1, z1 = all_adv_pc[6].T
x2, y2, z2 = test_X[6].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='blue')

In [ ]:
x1, y1, z1 = all_adv_pc[2].T
x2, y2, z2 = test_X[2].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='blue')